In [3]:
# Start coding here... 

## 1. BUSINESS GOALS

### 1.1 Merchandise and KPI metrics Formula 

- **Revenue** = count(unit) * base price * cost_per_unit / (1+VAT) 
- **Costs** = cost per unit * quantity * (1 + applicable tax) 
- **Margin** = (Revenue – Cost)/ Revenue 
- **AVG order value** = Total revenue / total number of orders/purchases 
- **Order volume** = Total number of orders/purchases 
- **AVG order size** = Total number of units sold / total number of purchases

### 1.2 Attributes for Exploratory Data Analysis

### purchase_data
- **PURCHASE_ID**: To calcualte KPIs for each purchase 
- **CURRENCY**: To identify which currency the purchase was made in 
- **COUNTRY**: The country of the venue where the purchase was made
- **VENUE_ID**: The unique identiﬁer of the venue 

### purchase_item_data 
- **PRODUCT_ID**: Identiﬁer for the item used. 
- **PURCHASE_ID**: The unique identiﬁer of the purchase
- **COUNT**: Thue number of items purchased 
- VENUE_ID: The unique identiﬁer of the venue
- **BASEPRICE**: The price of an item in local currency including value-added tax (VAT)
- **VAT_PERCENTAGE**: Value-added tax (VAT) percentage

### item_data
VENUE_ID: The unique identiﬁer of the venue
- **COST_PER_UNIT**: Cost of single unit of item in local currency excluding value-added tax (VAT)
- **COST_PER_UNIT_EUR**: Cost of single unit of item in Euro excluding value-added tax (VAT)
- **CURRENCY**: Currency for the item price
- **APPLICABLE_TAX_PERC**: Applicable tax percentage of the item
- **PRODUCT_ID**: Identiﬁer for the item used. 

## 2. EXPLORATORY DATA ANALYSIS

### 2.1 Purchase_data Table

- Focus on the column: **purchase_id, currency,venue_id** 
- During the given timeframe, the business has made 237 070 purchases from its users 
- There is no duplicate purchase has been recorded in this database.
- Each venue associates with only one currency

In [ ]:
-- Show purchase table 
SELECT * FROM purchase_data;

-- How many purchase has the company gained? Any purchase duplicate? 
SELECT COUNT(purchase_id) FROM purchase_data;

-- Check if any purchase is duplicate
SELECT purchase_id, COUNT(purchase_id) FROM purchase_data
GROUP BY purchase_id HAVING COUNT(purchase_id) > 1;

-- Since we need to calcualte KPI in the same currency unit, we will calcualte all KPI in one global currency EURO. Let's check what/how many currencies have been used in each venue
SELECT venue_id, COUNT(DISTINCT currency) as Num_of_cur FROM purchase_data
GROUP BY venue_id HAVING COUNT(DISTINCT currency) > 1;
--- The purchase value in each venue is calculated in ONLY one currency 


### 2.2 The purchase_item_data 

- This table provised us information about each purchase: which venue they have been made, which products in each, and the price of the sold items in the local currency. We have to use all attributes excep for the the first ordinal columns
- All quantity information is provided (no missing values)
- There are 245 products that have different base prices 
- Each product from each venue has only one VAT rate

In [ ]:
-- 2. EDA - purchase item data
SELECT * FROM purchase_item_data;

-- Any missing values in the 'count' column
SELECT SUM(CASE WHEN count is null THEN 1 ELSE 0 END) as quantity_NULL FROM purchase_item_data;
-- The quantity information is provided 

-- Check the baseprice of each product from the same venue_id 
SELECT purchase_id, venue_id, product_id, COUNT(DISTINCT baseprice)
FROM purchase_item_data 
GROUP BY purchase_id, venue_id, product_id HAVING COUNT(DISTINCT baseprice) > 1;
--- There are 245 products that have different base prices. 

-- Check the VAT_percentage of each product from the same venue_id 
SELECT purchase_id, venue_id, product_id, COUNT(DISTINCT vat_percentage)
FROM purchase_item_data 
GROUP BY purchase_id, venue_id, product_id HAVING COUNT(DISTINCT vat_percentage) > 1;
--- Each product from each venue has only one VAT rate. 

### 2.3 The item_data Table 

- This table provides us information of the item data (cost and tax) wihtout which we cannot calcualte the margin profits. Columns that will be used: **product_id**, **venue_id**, **currency**, **cost_per_unit**, **cost_per_unit_euro**, **applicable_tax_perc**
- There are **483094** rows missing in the column 'product_id'
- Among the products that have product_id available, there are **7493** products missing applicable tax percentage
- Among the products that have product_id available, there are **7492** products missing cost_per_unit


In [ ]:
---Show the item_data table
SELECT * FROM item_data;

-- Check missing value in the columns that will be used
-- How many product_id is missing?
SELECT SUM(CASE WHEN product_id is null THEN 1 ELSE 0 END) as product_id_NULL FROM item_data;
-- Applicable Tax Percentage is missing?
SELECT SUM(CASE WHEN applicable_tax_perc is null THEN 1 ELSE 0 END) as tax_perc_NULL FROM item_data
WHERE product_id IS NOT NULL;
-- Cost_per_unit_euro is missing?
SELECT SUM(CASE WHEN cost_per_unit_euro is null THEN 1 ELSE 0 END) as cost_per_unit_euro_NULL FROM item_data
WHERE product_id IS NOT NULL;

## 3. Data Preprocessing 

### 3.1 The purchase_item_data

- For the products that have different base prices, we will take the the Average of those values. 
- Since the base price is in the local currency, we have to convert them into the same currency - EURO

In [ ]:
CREATE VIEW purchase_item_view AS
SELECT
  -- the base price of the item is calcualted as the average of base prices in the same purchase and the same venue
pid.purchase_id, pid.venue_id, pid.product_id, vat_percentage, pd.currency, 
--- Convert the price in local currency to EURO
AVG(baseprice) OVER (PARTITION BY pid.venue_id, pid.purchase_id, pid.product_id)/ex_rate.rate AS baseprice_AVG_euro
FROM purchase_item_data AS pid
LEFT JOIN purchase_data AS pd ON pid.purchase_id = pd.purchase_id
LEFT JOIN currency_conv_rates AS ex_rate on pd.currency = ex_rate.currency
ORDER BY venue_id ASC, purchase_id ASC;

## 3.2 The item_data 

- For the products that missing some significant values (cost, applicable tax rate):
    - For the cost_per_unit_euro, I assumed that the company usually have the 50% markup percentage, meaning that for 1 euro cost unit , it will sell the product at 1.5 euros. 
    - For the missing applicable_tax_percentage, I will take the average of the tax rate in the same venue

In [ ]:
CREATE VIEW item_data_view AS(
SELECT DISTINCT idata.venue_id, idata.product_id, 
--- We only need information of the products recorded in the purchase data and thus RIGHT JOIN 
--- If the cost of item is missing, replace with the baseprice devisded by 1.5 (50% markup)
--- Replace the applicable_tax_perc is 15% if the information is missing 
(CASE WHEN idata.cost_per_unit_euro IS NULL THEN piv.baseprice_euro/1.5 ELSE cost_per_unit_euro END), 
(CASE WHEN idata.applicable_tax_perc IS NULL THEN 0.15 ELSE idata.applicable_tax_perc END)
FROM 
(SELECT venue_id, product_id, AVG(cost_per_unit_euro) OVER(PARTITION BY venue_id, product_id) AS cost_per_unit_euro, AVG(applicable_tax_perc) OVER(PARTITION BY venue_id, product_id) AS applicable_tax_perc
FROM item_data WHERE product_id IS NOT NULL) AS idata 
RIGHT JOIN (SELECT venue_id, product_id, AVG(baseprice_avg_euro) AS baseprice_euro FROM purchase_item_view GROUP BY venue_id, product_id) AS piv 
ON idata.venue_id = piv.venue_id AND idata.product_id = piv.product_id);

### 3.3 Profitability Table
- Create a table/view that is based on which different KPIs are calculated

In [ ]:
Create view calculation_view as 
SELECT  pd.time_delivered, pid.purchase_id, pid.product_id, pid.venue_id, pid.quantity, 
pid.quantity * pid.baseprice_avg_euro/(1+pid.vat_percentage/100) as Revenue,
quantity * idata.cost_per_unit_euro * (1+idata.applicable_tax_perc) as total_cost, pd.country, pd.currency
FROM purchase_item_view as pid
LEFT JOIN purchase_data AS pd ON pd.venue_id = pid.venue_id AND pd.purchase_id = pid.purchase_id
LEFT JOIN item_data_view as idata ON pid.product_id = idata.product_id AND pid.venue_id = idata.venue_id;

## 3. Data Analysis 

### Task 1 - Profitability of each purchase 

In [ ]:
--- Task 1 - Profitability of each purchase 
SELECT purchase_id, sum(revenue - total_cost) as Profitability
FROM calculation_view GROUP BY purchase_id; 

![image-5](image-5.png)


### Task 2 

In [ ]:
--- TOP 10 venues with the highest margins 
SELECT venue_id, (sum(revenue)- sum(total_cost))/ sum(revenue) *100 as margin, 
sum(quantity)/count(DISTINCT purchase_id) as AVG_order_size
FROM calculation_view GROUP BY venue_id ORDER BY margin DESC LIMIT 10;
--- TOP 5 countries for average order size 
SELECT country, sum(quantity)/count(DISTINCT purchase_id) as AVG_order_size
FROM calculation_view GROUP BY country ORDER BY AVG_order_size DESC LIMIT 5;

-- TOP 5 countries for average order value
SELECT country, sum(revenue)/count(DISTINCT purchase_id) AS AVG_order_value_eur
FROM calculation_view
GROUP BY country ORDER BY AVG_order_value_eur DESC LIMIT 5;

-- TOP 5 countries for order volumne 
SELECT country, count(DISTINCT purchase_id) as order_volume
FROM calculation_view GROUP BY country ORDER BY order_volume LIMIT 5;

--- Monthly cumulative Woltwide margin 
SELECT to_char(time_delivered, 'Month') AS month_2022, (sum(revenue)- sum(total_cost))/ 
sum(revenue) *100 as margin
FROM calculation_view GROUP BY month_2022;

![image-6](image-6.png)
![image-7](image-7.png)
![image-8](image-8.png)
![image-9](image-9.png)